# Imports

In [3]:
!pip install transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.8 MB/s eta 0:00:00


# HuggingFace

In [4]:
from huggingface_hub import notebook_login
notebook_login()

In [6]:
with open("/content/test.txt", 'r') as f:
  line = f.readlines()

In [7]:
line[0], line[-1]

('__label__benign के जरिए केन्द्र सरकार दिल्ली के विकास में रोड़े अटका रही है\n',
 '__label__abusive जो गाँड से सुई में धागा पिरो दे वो राजपूत\n')

In [8]:
len(line)

397

# Checking if there are more than 2 labels

In [9]:
for sent in line:
  label_sent = sent.split(' ')
  if label_sent[2].find('label') > -1 :
      print('found')
      print(sent)

# Separating labels from the texts

In [10]:
def seperate_labels(line_list: list) -> tuple[list,list]:
  """
  A function to seperate the labels from the sentences if exists

  Parameters:
  -----------
  line_list: list of lines with labels

  Returns:
  ----------
  tuple(list, list): 2 lists, first is labels, and second is corresponding sentences
  """
  # labels and sentences container
  labels = []
  sentences = []

  # seperating from each line
  for sent in line_list:
    label_sent = sent.split(' ')

    # if there is no label skip
    if label_sent[0].find('label') < 0 :
      continue

    # getting the first label
    label = label_sent[0]

    # if there is one more
    if label_sent[1].find('label') > -1 :
      label = label + label_sent[1]

      # if there are 3
      if label_sent[2].find('label') > -1:
        label = label + label_sent[2]
        sentences.append(" ".join(label_sent[3:]))
        labels.append(label)

      else:
         sentences.append(" ".join(label_sent[2:]))
         labels.append(label)


    # Append the sentence and the label to the respective container
    else:
      sentences.append(" ".join(label_sent[1:]))
      labels.append(label)

  return labels, sentences

# Checking the labels and sentences

In [11]:
labels, sents = seperate_labels(line)

In [12]:
len(labels), len(sents)

(397, 397)

In [13]:
labels[-1], sents[-1]

('__label__abusive', 'जो गाँड से सुई में धागा पिरो दे वो राजपूत\n')

# Checking if there are yet more labels

In [16]:
for i in range(len(sents)):
  if sents[i].find('label') > -1:
    print(f'found at {i}')
    print(sents[i])

# Checking the unqiue labels

In [17]:
categories = set()
for label in labels:
  categories.add(label)
categories

{'__label__abusive',
 '__label__abusive__label__hatespeech',
 '__label__abusive__label__humor',
 '__label__benign',
 '__label__benignपिच',
 '__label__hatespeech__label__abusive',
 '__label__humor__label__abusive'}

# Cleaning the words from labels

In [18]:
for i in range(len(labels)):
  target = ["__label__benignपिच"]
  if labels[i] in target:
    print(f"{i}{labels[i]}{sents[i]}")

385__label__benignपिचको समझने के लिये पाकिस्तान को इतने रन बनाने देने की क्या जरूरत थी ।।। साला पिच क्यूरेटर से ही पूछ लेते ।।।	  



In [20]:
print(labels[385])
print(labels[385].find("पिच"))

__label__benignपिच
15


In [22]:
print(labels[385][:15])

__label__benign


In [23]:
labels[385] = labels[385][:15]

In [24]:
labels[385]

'__label__benign'

# Final check for cleaned unique labels

In [25]:
categories = set()
for label in labels:
  categories.add(label)
categories

{'__label__abusive',
 '__label__abusive__label__hatespeech',
 '__label__abusive__label__humor',
 '__label__benign',
 '__label__hatespeech__label__abusive',
 '__label__humor__label__abusive'}

# Getting the dataframe and csv

In [45]:
import pandas as pd
data = pd.DataFrame()
data["label"] = labels
data["Sentences"] = sents

In [46]:
data

,label,Sentences
0,__label__benign,के जरिए केन्द्र सरकार दिल्ली के विकास में रोड़...
1,__label__benign,बीजेपी को तो कभी मुसलमानो ने अपना माना नहीक्या...
2,__label__benign,बी जे पी सरकार मे शिक्षा मित्रो की दुर्गति हो ...
3,__label__benign,अब लड़ाई कॉंग्रेस और बीजेपी में नहीं रही अब ल...
4,__label__benign,राजस्थान उपचुनाव नतीजे राजस्थान में कांग्रेस ...
...,...,...
392,__label__abusive,दोस्तों मैं जानती हूँ की इस समय आपका हाथ आपके ...
393,__label__abusive,कुते वफादार होते हैं पर तुम जैसे सुअर नही होते...
394,__label__abusive,भेंचो जीजा अपने लण्ड पर क्रांति लिखने में बिजी...
395,__label__abusive,आज पप्पू और चोदू दोनों साथ साथ होंगे मतलब\t \n


In [29]:
data.to_csv('abusive_tweet_hindi_test.csv', index=False)

# Another csv cleaned with abusive or hatespeech merged

In [47]:
import numpy as np
encoded_labels = np.full((len(sents),), fill_value = "Null")

for i in range(len(labels)):
  if labels[i].find("hatespeech") > -1:
    encoded_labels[i] = 'HOF'
  elif labels[i].find("abusive") > -1:
    encoded_labels[i] = 'HOF'
  else:
    encoded_labels[i] = 'NOT'

In [48]:
encoded_labels

array(['NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT', 'NOT',
       'NOT', 'NOT',

# DataFrame and CSV

In [49]:
import pandas as pd
data = pd.DataFrame()
data["label"] = encoded_labels
data["Sentences"] = sents

In [50]:
data.to_csv('abusive_encoded_hindi_test.csv', index=False)

# Loading the dataset

In [51]:
view_data = pd.read_csv("/content/abusive_encoded_hindi_test.csv")
view_data

,label,Sentences
0,NOT,के जरिए केन्द्र सरकार दिल्ली के विकास में रोड़...
1,NOT,बीजेपी को तो कभी मुसलमानो ने अपना माना नहीक्या...
2,NOT,बी जे पी सरकार मे शिक्षा मित्रो की दुर्गति हो ...
3,NOT,अब लड़ाई कॉंग्रेस और बीजेपी में नहीं रही अब ल...
4,NOT,राजस्थान उपचुनाव नतीजे राजस्थान में कांग्रेस ...
...,...,...
392,HOF,दोस्तों मैं जानती हूँ की इस समय आपका हाथ आपके ...
393,HOF,कुते वफादार होते हैं पर तुम जैसे सुअर नही होते...
394,HOF,भेंचो जीजा अपने लण्ड पर क्रांति लिखने में बिजी...
395,HOF,आज पप्पू और चोदू दोनों साथ साथ होंगे मतलब\t \n


# Numerical encoding

In [52]:
mappings = {'HOF': 1, 'NOT': 0}

In [53]:
encoded_list = []
for rows in data["label"].items():
  encoded_list.append(mappings[rows[1]])

# Adding encodings to the data

In [54]:
data["encoded_labels"] = encoded_list

In [55]:
data

,label,Sentences,encoded_labels
0,NOT,के जरिए केन्द्र सरकार दिल्ली के विकास में रोड़...,0
1,NOT,बीजेपी को तो कभी मुसलमानो ने अपना माना नहीक्या...,0
2,NOT,बी जे पी सरकार मे शिक्षा मित्रो की दुर्गति हो ...,0
3,NOT,अब लड़ाई कॉंग्रेस और बीजेपी में नहीं रही अब ल...,0
4,NOT,राजस्थान उपचुनाव नतीजे राजस्थान में कांग्रेस ...,0
...,...,...,...
392,HOF,दोस्तों मैं जानती हूँ की इस समय आपका हाथ आपके ...,1
393,HOF,कुते वफादार होते हैं पर तुम जैसे सुअर नही होते...,1
394,HOF,भेंचो जीजा अपने लण्ड पर क्रांति लिखने में बिजी...,1
395,HOF,आज पप्पू और चोदू दोनों साथ साथ होंगे मतलब\t \n,1


# Drop the initial NOT and HOF

In [56]:
data = data.drop(data[["label"]], axis = 1)

In [57]:
data

,Sentences,encoded_labels
0,के जरिए केन्द्र सरकार दिल्ली के विकास में रोड़...,0
1,बीजेपी को तो कभी मुसलमानो ने अपना माना नहीक्या...,0
2,बी जे पी सरकार मे शिक्षा मित्रो की दुर्गति हो ...,0
3,अब लड़ाई कॉंग्रेस और बीजेपी में नहीं रही अब ल...,0
4,राजस्थान उपचुनाव नतीजे राजस्थान में कांग्रेस ...,0
...,...,...
392,दोस्तों मैं जानती हूँ की इस समय आपका हाथ आपके ...,1
393,कुते वफादार होते हैं पर तुम जैसे सुअर नही होते...,1
394,भेंचो जीजा अपने लण्ड पर क्रांति लिखने में बिजी...,1
395,आज पप्पू और चोदू दोनों साथ साथ होंगे मतलब\t \n,1


In [59]:
data.to_csv('encoded_labels_hindi_tweets_test.csv', index=False)